In [4]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings("ignore")

In [5]:
# adapter le chemin vers le fichier CSV nettoyé produit par le notebook 01_xiaosong_text_clean.ipynb
df = pd.read_csv("rakuten_text_train_v1.csv")


In [6]:
def safe_str(x):
    if isinstance(x, str):
        return x
    if pd.isna(x):
        return ""
    return str(x)

# Normalisation des deux colonnes texte principales
df["designation_cleaned"] = df["designation_cleaned"].fillna("").apply(safe_str)
df["description_cleaned"] = df["description_cleaned"].fillna("").apply(safe_str)

# Colonne texte fusionnée (titre + description) pour la variante "texte concaténé"
df["text_all"] = (df["designation_cleaned"] + " " + df["description_cleaned"]).str.strip()


In [7]:
# Features structurales sur le texte

UNIT_PATTERN = re.compile(r"\b\d+\s*(cm|mm|kg|g|ml|l|m)\b", flags=re.IGNORECASE)
MULT_PATTERN = re.compile(r"\bx\s*\d+\b|\b\d+\s*x\b", flags=re.IGNORECASE)
DIGIT_PATTERN = re.compile(r"\d")

def structural_stats(s: str) -> dict:
    """Calcule des indicateurs simples de structure."""
    s = safe_str(s)
    tokens = s.split()
    length_char = len(s)
    length_tokens = len(tokens)
    
    num_digits = len(DIGIT_PATTERN.findall(s))
    num_units = len(UNIT_PATTERN.findall(s))
    num_mult = len(MULT_PATTERN.findall(s))
    
    return {
        "len_char": length_char,
        "len_tokens": length_tokens,
        "num_digits": num_digits,
        "num_units": num_units,
        "num_mult_pattern": num_mult,
    }

# Application sur le titre et la description
for col in ["designation_cleaned", "description_cleaned"]:
    stats_series = df[col].apply(structural_stats)
    stats_df = pd.DataFrame(list(stats_series))
    for c in stats_df.columns:
        df[f"{col}_{c}"] = stats_df[c]

# Aperçu de quelques features de longueur
df[
    [
        "designation_cleaned_len_char",
        "designation_cleaned_len_tokens",
        "description_cleaned_len_char",
        "description_cleaned_len_tokens",
    ]
].head()


,designation_cleaned_len_char,designation_cleaned_len_tokens,description_cleaned_len_char,description_cleaned_len_tokens
0,80,10,0,0
1,161,24,0,0
2,72,10,546,70
3,57,7,0,0
4,13,2,127,18


In [8]:
# TF-IDF pour le titre (designation)
tfidf_title = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1, 3),
    min_df=5,
    max_df=0.8,
    lowercase=False,
    tokenizer=str.split,
)

X_tfidf_title = tfidf_title.fit_transform(df["designation_cleaned"])
print("TF-IDF titre - forme :", X_tfidf_title.shape)

# TF-IDF pour la description
tfidf_desc = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1, 3),
    min_df=5,
    max_df=0.8,
    lowercase=False,
    tokenizer=str.split,
)

X_tfidf_desc = tfidf_desc.fit_transform(df["description_cleaned"])
print("TF-IDF description - forme :", X_tfidf_desc.shape)

# TF-IDF sur le texte fusionné (titre + description)
tfidf_all = TfidfVectorizer(
    max_features=40000,
    ngram_range=(1, 3),
    min_df=5,
    max_df=0.8,
    lowercase=False,
    tokenizer=str.split,
)

X_tfidf_all = tfidf_all.fit_transform(df["text_all"])
print("TF-IDF texte fusionné - forme :", X_tfidf_all.shape)


TF-IDF titre - forme : (84916, 20000)
TF-IDF description - forme : (84916, 30000)
TF-IDF texte fusionné - forme : (84916, 40000)


In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Standardisation des features numériques
num_scaler = StandardScaler(with_mean=False)  # with_mean=False pour matrices creuses

# Colonnes numériques dérivées des stats structurales
meta_cols = [
    c for c in df.columns
    if c.startswith("designation_cleaned_")
    or c.startswith("description_cleaned_")
]

print("Nombre de colonnes numériques (features structurales) :", len(meta_cols))

# ==============================
# 1) Préprocesseur A : titre / description séparés
#    (on pourra jouer sur les poids des deux blocs TF-IDF)
# ==============================
preprocess_split = ColumnTransformer(
    transformers=[
        ("tfidf_title", tfidf_title, "designation_cleaned"),
        ("tfidf_desc", tfidf_desc, "description_cleaned"),
        ("num", num_scaler, meta_cols),
    ],
    remainder="drop",
)

# Exemple de pondération manuelle : on donne plus de poids au titre
preprocess_split.set_params(
    transformer_weights={
        "tfidf_title": 2.0,   # poids du bloc TF-IDF du titre
        "tfidf_desc": 1.0,   # poids du bloc TF-IDF de la description
        "num": 1.0,          # poids des features numériques
    }
)

# ==============================
# 2) Préprocesseur B : texte fusionné (titre + description)
# ==============================
preprocess_merged = ColumnTransformer(
    transformers=[
        ("tfidf_all", tfidf_all, "text_all"),
        ("num", num_scaler, meta_cols),
    ],
    remainder="drop",
)

# Modèle de base (le même pour les deux pipelines)
log_reg = LogisticRegression(
    C=2.0,
    max_iter=1000,
    class_weight="balanced",
    solver="saga",
    n_jobs=-1,
)

# Pipeline A : texte séparé (titre / description)
clf_split = Pipeline(
    steps=[
        ("preprocess", preprocess_split),
        ("model", log_reg),
    ]
)

# Pipeline B : texte fusionné
clf_merged = Pipeline(
    steps=[
        ("preprocess", preprocess_merged),
        ("model", log_reg),
    ]
)

print("Pipeline (texte séparé) :", clf_split)
print("Pipeline (texte fusionné) :", clf_merged)


Nombre de colonnes numériques (features structurales) : 10
Pipeline (texte séparé) : Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformer_weights={'num': 1.0,
                                                        'tfidf_desc': 1.0,
                                                        'tfidf_title': 2.0},
                                   transformers=[('tfidf_title',
                                                  TfidfVectorizer(lowercase=False,
                                                                  max_df=0.8,
                                                                  max_features=20000,
                                                                  min_df=5,
                                                                  ngram_range=(1,
                                                                               3),
                                                                  tokenizer=<method 'split' of 'str' objects>),

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

# Construction de X avec toutes les colonnes utiles
X = df[["designation_cleaned", "description_cleaned", "text_all"] + meta_cols]
y = df["prdtypecode"].values

# Split entraînement / validation
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print("Taille X_train :", X_train.shape)
print("Taille X_valid :", X_valid.shape)

# =========================
# 1) Modèle A : texte séparé
# =========================
print("\nEntraînement du modèle A (titre / description séparés, pondérés)...")
clf_split.fit(X_train, y_train)

y_pred_split = clf_split.predict(X_valid)
f1_split = f1_score(y_valid, y_pred_split, average="weighted")

print(f"\n[Modèle A] Weighted F1 (validation) : {f1_split:.4f}\n")
print("[Modèle A] Classification report :\n")
print(classification_report(y_valid, y_pred_split))

# =========================
# 2) Modèle B : texte fusionné
# =========================
print("\nEntraînement du modèle B (texte fusionné)...")
clf_merged.fit(X_train, y_train)

y_pred_merged = clf_merged.predict(X_valid)
f1_merged = f1_score(y_valid, y_pred_merged, average="weighted")

print(f"\n[Modèle B] Weighted F1 (validation) : {f1_merged:.4f}\n")
print("[Modèle B] Classification report :\n")
print(classification_report(y_valid, y_pred_merged))

# Comparaison simple des scores
print("\nRésumé des scores (validation) :")
print(f" - Modèle A (séparé)  : {f1_split:.4f}")
print(f" - Modèle B (fusionné) : {f1_merged:.4f}")
print(f" - Différence (A - B)  : {f1_split - f1_merged:.4f}")


Taille X_train : (67932, 13)
Taille X_valid : (16984, 13)

Entraînement du modèle A (titre / description séparés, pondérés)...

[Modèle A] Weighted F1 (validation) : 0.8067

[Modèle A] Classification report :

              precision    recall  f1-score   support

          10       0.42      0.66      0.51       623
          40       0.75      0.65      0.69       502
          50       0.79      0.82      0.80       336
          60       0.85      0.83      0.84       166
        1140       0.76      0.81      0.78       534
        1160       0.91      0.92      0.92       791
        1180       0.57      0.67      0.61       153
        1280       0.76      0.56      0.64       974
        1281       0.60      0.59      0.60       414
        1300       0.86      0.90      0.88      1009
        1301       0.91      0.96      0.93       161
        1302       0.77      0.79      0.78       498
        1320       0.80      0.76      0.78       648
        1560       0.86      0.79

In [11]:
from copy import deepcopy

# Liste de pondérations à tester pour (titre, description)
weight_grid = [
    (1.0, 1.0),
    (2.0, 1.0),
    (3.0, 1.0),
    (1.0, 2.0),
]

results = []

for w_title, w_desc in weight_grid:
    print("\n" + "=" * 50)
    print(f"Pondérations : titre={w_title}, description={w_desc}")
    print("=" * 50)
    
    # Copie du préprocesseur pour ne pas écraser l'original
    preprocess_w = deepcopy(preprocess_split)
    preprocess_w.set_params(
        transformer_weights={
            "tfidf_title": w_title,
            "tfidf_desc": w_desc,
            "num": 1.0,
        }
    )
    
    clf_w = Pipeline(
        steps=[
            ("preprocess", preprocess_w),
            ("model", log_reg),
        ]
    )
    
    clf_w.fit(X_train, y_train)
    y_pred_w = clf_w.predict(X_valid)
    f1_w = f1_score(y_valid, y_pred_w, average="weighted")
    
    print(f"Weighted F1 (validation) : {f1_w:.4f}")
    results.append((w_title, w_desc, f1_w))

print("\nRésumé pondérations / scores :")
for w_title, w_desc, f1_w in results:
    print(f" - (titre={w_title}, description={w_desc}) -> F1={f1_w:.4f}")



Pondérations : titre=1.0, description=1.0
Weighted F1 (validation) : 0.7723

Pondérations : titre=2.0, description=1.0
Weighted F1 (validation) : 0.8067

Pondérations : titre=3.0, description=1.0
Weighted F1 (validation) : 0.8159

Pondérations : titre=1.0, description=2.0
Weighted F1 (validation) : 0.7808

Résumé pondérations / scores :
 - (titre=1.0, description=1.0) -> F1=0.7723
 - (titre=2.0, description=1.0) -> F1=0.8067
 - (titre=3.0, description=1.0) -> F1=0.8159
 - (titre=1.0, description=2.0) -> F1=0.7808


In [12]:
# from sklearn.model_selection import GridSearchCV


# param_grid = {
#     "model__C": [0.5, 1.0, 2.0],
#     "preprocess__title_tfidf__max_features": [10000, 20000],
#     "preprocess__desc_tfidf__max_features": [20000, 30000],
# }

# grid = GridSearchCV(
#     estimator=clf_pipeline,
#     param_grid=param_grid,
#     scoring="f1_weighted",
#     cv=3,
#     n_jobs=-1,
#     verbose=2,
# )

# print("Lancement de la GridSearch (peut être un peu long)...")
# grid.fit(X_train, y_train)

# print("\nMeilleurs paramètres trouvés :", grid.best_params_)
# print("Meilleur score (F1 pondéré, CV) :", grid.best_score_)

# # Évaluation du meilleur modèle sur le jeu de validation
# best_model = grid.best_estimator_
# y_pred_best = best_model.predict(X_valid)
# best_f1 = f1_score(y_valid, y_pred_best, average="weighted")

# print(f"\nWeighted F1 du meilleur modèle sur validation : {best_f1:.4f}\n")
# print("Classification report du meilleur modèle :\n")
# print(classification_report(y_valid, y_pred_best))
